# Ludii Translator
- Translate Ludii to GDL: translated Tic-Tac-Toe currently

In [2]:
import re

## Constant

In [1]:
CELL_MART = ['x', 'o']
TERMINAL_CONDITION = ['WIN', 'DRAW', 'LOSS']
PLAYERREWARD = {"WIN":100, "DRAW":50, "LOSS":0} 

## Class Translater

In [3]:
class Translator:
    def __init__(self, fileName):
        self.info = loadLudii(fileName)
        self.players = self.extractPlayers()
        self.board = self.extractBoard()
        self.end = self.extractEnd()
        
    
    
    def extractPlayers(self):
        """
        extract players information
        """
        list_pl = []
        for var in self.info['equipment']:
            gName = re.search(".*(P\d*)", var)
            if gName:
                list_pl.append(gName.group(1))
        return list_pl
    
    
    def extractBoard(self):
        """
        extract board information
        """
        for var in self.info['equipment']:
            board = re.search("board \((.*)\){2}", var)
            if board:
                return board.group(1)
    
    
    def extractEnd(self):
        list_end = []
        for var in self.info['rules']:
            if "(end" in var:
                list_end.append(var)
        return list_end
    
    
    def translating(self):
        with open(self.info['game']+'.gdl', 'w+') as fp:
            
            # translating game name
            fp.write(translateGame(self.info['game']))
            
            # translating players
            fp.write(translatePlayers(self.players))
            
            # translating game type
            # turn-based game
            fp.write(translateMode(self.players))
            
            # translating board
            fp.write(translateBoard(self.board, self.players[0]))
            
            # translating winCondition
            fp.write(translatedWinCondition(self.end, self.board))
            
            # translating legal action
            fp.write(translatedLegalAction(self.players, self.board))
            
            # translating move & update
            fp.write(translatedMoveAndUpdate(self.players))
            
            
            #translating end (reward & terminal)
            fp.write(translatedEnd(self.end, self.players))
        pass

## class BoardConvert
- process keyword 'board'
- type and size

In [4]:
class BoardConvert:
    def __init__(self, boardType, boardSize, initPlayer):
        self.boardType = boardType
        self.boardSize = boardSize
        self.initPlayer = initPlayer
    
    
    def convert(self):
        context = ";"*80 + "\n" + ";; Initial State\n" + ";"*80 + "\n\n"
        
        # translating square type
        if self.boardType == "square":
            for i in range(1, self.boardSize+1):
                for j in range(1, self.boardSize+1):
                    context += f"(init (cell {i} {j} b))\n"
            context += f"(init (control {self.initPlayer}))\n\n"
        return context

## Class Terminal
- process keywork 'end'
- win condition

In [5]:
class Terminal:    
    def __init__(self, endInfor, players, winCon, boardInfo):
        self.win_Dict = self.decompositionEnd(endInfor)
        self.players = players
        self.winCon = winCon
        self.boardInfo = boardInfo
    
    
    def decompositionEnd(self, Infor):
        if Infor:
            end_dict = {}
            for var in Infor:
                endVar = re.search("\(end \(if \((.*)\) \((.*)\)\)\)" ,var)
                if endVar:
                    end_dict[endVar.group(1)] = endVar.group(2)
            return end_dict
        else:
            return None
        
        
    def convertEndInfo(self):
        context = ";"*80 + "\n" + ";;; Reward\n" + ";"*80 + "\n\n"
        for key, var in self.win_Dict.items():
            if "is Line" in key and "result Mover Win" in var:
                # reward
                for i in range(len(TERMINAL_CONDITION)):
                        for j in range(len(self.players)):
                            if PLAYERREWARD[TERMINAL_CONDITION[i]] == 100:
                                context += f"(<= (goal {self.players[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) (line {CELL_MART[j]}))\n"
                            elif PLAYERREWARD[TERMINAL_CONDITION[i]] == 0:
                                context += f"(<= (goal {self.players[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) (line {CELL_MART[j-1]}))\n"
                            else:
                                context += f"(<= (goal {self.players[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) "
                                for z in range(len(self.players)):
                                    context += f"(not (line {CELL_MART[z]})) "c
                                context += "(not open))\n"
                
                
                # terminal
                context += "\n"+";"*80 + "\n" + ";;; End\n" + ";"*80 + "\n\n"
                for i in range(len(TERMINAL_CONDITION)):
                    if i < len(self.players):
                        context += f"(<= terminal(line {CELL_MART[i]}))\n"
                    else:
                        context += "(<= terminal(not open))\n\n" # draw
        return context



    def convertWinInfo(self, mode="Line"):
        if mode == "Line" and "square" in self.boardInfo:
            
            # row
            contextRow = "(<= (row ?m ?x)"
            contextCol = "(<= (column ?n ?x)"
            contextDia1 = "(<= (diagonal ?x)"
            contextDia2 = "(<= (diagonal ?x)"
            for i in range(int(self.boardInfo[7])):
                contextRow += f" (true (cell ?m {i+1} ?x))"
                contextCol += f" (true (cell {i+1} ?n ?x))"
                contextDia1 += f" (true (cell 1 {i+1} ?x))"
                contextDia2 += f" (true (cell 1 {int(self.boardInfo[7])-i} ?x))"
            contextRow += "\n"
            contextCol += "\n"
            contextDia1 += "\n"
            contextDia2 += "\n"
            
            context = contextRow + contextCol + contextDia1 + contextDia2 + "\n"
            context += "(<= (line ?x) (row ?m ?x))\n(<= (line ?x) (column ?m ?x))\n(<= (line ?x) (diagonal ?x))\n\n"
        return context
    
    

## Class Utils

In [6]:
def loadLudii(fileName):
    '''
    loading Ludii and store the information as dict type
    :param fileName: the filename of game
    return a dict that contain the information of correspond game
    '''
    info = "".join(readfile(fileName).split("//")[:-1]).split("\n")
    
    info_dict = {}
    for i in range(len(info)):
        if "game" in info[i]:
            info_dict["game"] = info[i].strip()[7:-1]

        elif "players" in info[i]:
            info_dict["players"] = int(info[i].strip()[9:-1])

        elif "equipment" in info[i]:
            list_eq = []
            i += 1
            while "})" not in info[i]:
                list_eq.append(info[i].strip())
                i += 1
            info_dict["equipment"] = list_eq

        elif "rules" in info[i]:
            list_ru = []
            i += 1
            while len(info[i].strip()) > 1:
                list_ru.append(info[i].strip())
                i += 1
            info_dict["rules"] = list_ru
    return info_dict


def readfile(fileName):
    '''
    read the file
    :param fileName: the name of file
    return the context of file
    '''
    with open(fileName) as fp:
        return fp.read()


def translateGame(gameName):
    '''
    translate the game name from ludii to GDL
    :param gameName: the name of game
    return string-based GDL
    '''
    context = ";"*80 + "\n" + f";;; {gameName}\n" + ";"*80 + "\n\n"
    return context


def translatePlayers(players):
    '''
    translate the players description from ludii to GDL
    :param players: a list that including the name of player
    return string-based GDL
    '''
    context = ";"*80 + "\n" + ";; Roles\n" + ";"*80 + "\n\n"
#     for i in range(players):
    for var in players:
        context += f"(role {var})\n"
    context += "\n"
    return context


def translateMode(players):
    """
    translate the mode
    :param players: the list of player
    return string-based GDL that including the turn mode
    """
    context = ";"*80 + "\n" + ";; Game Mode\n" + ";"*80 + "\n\n"
    for i in range(len(players)):
        context += f"(<= (next (control {players[i]})) (true (control {players[(i+1)%len(players)]})))\n"
    context += "\n"
    return context


def translateBoard(board, initPlayer):
    """
    translate the board description from ludii to GDL
    it contains init each cell and player
    :param board: the information of board
    :param initPlayer: the player will be controlled first
    return string-based GDL that including init cell & players
    """
    bType, bSize = board.split(" ")
    return BoardConvert(bType, int(bSize), initPlayer).convert()


def translatedEnd(endInfo, playersInfo):
    """
    translate the end description from ludii to GDL
    *keyword 'end' includes reward & terminal in GDL
    :param endInfo: end information
    :param playersInfo: players Information
    return string-based GDL that including reward & terminal
    """
    return Terminal(endInfo, playersInfo, None, None).convertEndInfo()
    

def translatedWinCondition(winCons, board):
    """
    translate the win-condition(Line) from Ludii to GDL
    :param winCons: end information
    :param board: board information
    """
    context = ";"*80 + "\n" + ";; Win Condition\n" + ";"*80 + "\n\n"
    
    for winCon in winCons:
        if "is Line" in winCon:
            context += Terminal(None, None, winCon, board).convertWinInfo(mode="Line")
    return context


def translatedLegalAction(players, board):
    """
      translate the legal action from Ludii to GDL
      :param players: players information
      :param board: board information
      return string-based GDL that including legal-action information
    """
    context = ";"*80 + "\n" + ";; Legal Action\n" + ";"*80 + "\n\n"
    if "square" in board:
        for i in range(len(players)):
            context += f"(<= (legal {players[(i+1)%len(players)]} noop) (true (control {players[i]})))\n"
        context += "(<= (legal ?w (mark ?x ?y)) (true (cell ?x ?y b)) (true (control ?w)))\n\n"
    return context


def translatedMoveAndUpdate(players):
    """
        (<= (next (cell ?m ?n x))
        (does xplayer (mark ?m ?n))
        (true (cell ?m ?n b)))

        (<= (next (cell ?m ?n o))
        (does oplayer (mark ?m ?n))
        (true (cell ?m ?n b)))

        (<= (next (cell ?m ?n ?w))
        (true (cell ?m ?n ?w))
        (distinct ?w b))

        (<= (next (cell ?m ?n b))
        (does ?w (mark ?j ?k))
        (true (cell ?m ?n b))
        (or (distinct ?m ?j) (distinct ?n ?k)))
    """
    context = ";"*80 + "\n" + ";;  Move & Update\n" + ";"*80 + "\n\n"
    for i in range(len(players)):
        context += f"(<= (next (cell ?m ?n {CELL_MART[i]})) (does {players[i]} (mark ?m ?n)) (true (cell ?m ?n b)))\n"
    
    # update mark cell
    context += "(<= (next (cell ?m ?n ?w)) (true (cell ?m ?n ?w)) (distinct ?w b))\n"
    context += "(<= (next (cell ?m ?n b)) (does ?w (mark ?j ?k)) (true (cell ?m ?n b)) (or (distinct ?m ?j) (distinct ?n ?k)))\n\n"
    return context
    
    

# test

## read file

In [9]:
trl = Translator('Tic-Tac-Toe.lud')

ValueError: invalid literal for int() with base 10: '{(player N) (player S)}'

In [8]:
print(trl.info)
print(trl.players)
print(trl.board)
print(trl.end)

{'game': 'Tic-Tac-Toe', 'players': 2, 'equipment': ['(board (square 3))', '(piece "Disc" P1)', '(piece "Cross" P2)'], 'rules': ['(play (move Add (to (sites Empty))))', '(end (if (is Line 3) (result Mover Win)))']}
['P1', 'P2']
square 3
['(end (if (is Line 3) (result Mover Win)))']


In [148]:
trl.translating()

In [315]:
print(Terminal(trl.end, trl.players).convert())

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(<= (goal P1 100) (line x))
(<= (goal P2 100) (line o))
(<= (goal P1 50) (not (line x)) (not (line o)) (not open))
(<= (goal P2 50) (not (line x)) (not (line o)) (not open))
(<= (goal P1 0) (line o))
(<= (goal P2 0) (line x))

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(<= terminal(line x))
(<= terminal(line o))
(<= terminal(not open))




In [269]:
print(ee.win_Dict)

{'is Line 3': 'result Mover Win'}
